In [ ]:
import pandas as pd
pd.get_option("display.max_rows")

train = pd.read_csv('./brist1d/train.csv')
test = pd.read_csv('./brist1d/test.csv')

<ipython-input-6-12f8329d0f22>:4: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('/content/drive/MyDrive/KSVA_AI/data/brist1d/train.csv')


In [ ]:
train.head()

,id,p_num,time,bg-5:55,bg-5:50,bg-5:45,bg-5:40,bg-5:35,bg-5:30,bg-5:25,...,activity-0:40,activity-0:35,activity-0:30,activity-0:25,activity-0:20,activity-0:15,activity-0:10,activity-0:05,activity-0:00,bg+1:00
0,p01_0,p01,06:10:00,NaN,NaN,9.6,NaN,NaN,9.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.4
1,p01_1,p01,06:25:00,NaN,NaN,9.7,NaN,NaN,9.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.8
2,p01_2,p01,06:40:00,NaN,NaN,9.2,NaN,NaN,8.7,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.5
3,p01_3,p01,06:55:00,NaN,NaN,8.7,NaN,NaN,8.4,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.8
4,p01_4,p01,07:10:00,NaN,NaN,8.4,NaN,NaN,8.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7


In [ ]:
# ---------- 열 카테고리 분류 ----------
def column_category(col):
    """
    Returns high‑level category for a column
    """
    if "-" in col:
        return col.split("-")[0]  # bg, insulin, carbs, hr, steps, cals, activity
    return col  # id, p_num, time, bg+1:00

col_cats = {col: column_category(col) for col in train.columns}

# ---------- 카테고리별 열 개수 ----------
cat_counts = (
    pd.Series(col_cats)
      .value_counts()
      .rename_axis("category")
      .reset_index(name="n_columns")
      .sort_values("n_columns", ascending=False)
)

# ---------- 결측(NA) 비율 요약 ----------
na_summary = (
    train.isna()
      .groupby(col_cats, axis=1)
      .mean()      # fraction NaN per column group
      .T
      .reset_index()
      .rename(columns={"index":"category"})
)

# ---------- bg+1:00 기술통계 ----------
target_stats = train["bg+1:00"].describe().to_frame("bg+1:00").T


<ipython-input-9-2b31ee1cf40a>:24: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  .groupby(col_cats, axis=1)


In [ ]:
cat_counts

,category,n_columns
0,carbs,72
1,insulin,72
2,bg,72
3,steps,72
4,cals,72
5,activity,72
6,hr,72
7,time,1
8,p_num,1
9,id,1


# 카테고리별 변수 의미

| 카테고리    | 개수 | 설명                                                                                                                                                                     |
|-------------|-----:|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **bg**      |   72 | **혈당(Blood Glucose)**<br>과거 **0~6시간(5분 간격 총 72포인트)** 전 시점별 혈당(mmol/L) 값입니다. CGM(연속혈당측정기)에서 5분마다 기록된 혈당을 lag 변수로 나열한 것입니다.      |
| **insulin** |   72 | **인슐린 투여량**<br>과거 **0~6시간(5분 간격)** 동안 각 5분 구간에 투여된 인슐린 총량(U)입니다. 인슐린 펌프·자가 주사 기록을 5분 단위로 누적한 lag 변수입니다.                |
| **carbs**   |   72 | **탄수화물 섭취량**<br>과거 **0~6시간(5분 간격)** 동안 각 5분 구간에 섭취한 탄수화물(g) 총량입니다. 참가자 자가 보고 식사·간식 데이터를 lag 변수로 구성했습니다.               |
| **hr**      |   72 | **심박수(Heart Rate)**<br>과거 **0~6시간(5분 간격)** 구간별 평균 심박수(bpm)입니다. 스마트워치 등 웨어러블 기기에서 기록된 5분 단위 심박수 lag 변수입니다.                     |
| **steps**   |   72 | **걸음 수(Steps)**<br>과거 **0~6시간(5분 간격)** 동안 걸은 걸음 수입니다. 스마트워치·폰 걸음계 데이터를 5분 구간 누적으로 lag 변수화했습니다.                                 |
| **cals**    |   72 | **칼로리 소모(Calories)**<br>과거 **0~6시간(5분 간격)** 동안 소모된 칼로리(kcal)입니다. 웨어러블 기기의 활동 추정치를 5분 단위로 lag 변수로 나열했습니다.                     |
| **activity**|   72 | **활동 종류(Activity)**<br>과거 **0~6시간(5분 간격)** 구간별 자가 보고 활동명(예: `walking`, `running`, `sleep`)입니다. 문자형 카테고리 lag 변수로 구성되었습니다.                |
| **time**    |    1 | **관찰 시각(Time of day)**<br>이 행이 기록된 하루 중 시각(HH:MM:SS)입니다. 날짜는 없고 시간만 있어, 하루 내 패턴 분석에 사용됩니다.                                      |
| **p_num**   |    1 | **참가자 번호(Participant ID)**<br>각 사용자를 구분하는 고유 코드입니다.                                                                    |
| **id**      |    1 | **행 고유 ID**<br>`p_num`과 해당 참가자 내 순번(count)이 결합된 값으로, 모든 행을 유니크하게 식별합니다.                                           |
| **bg+1:00** |    1 | **타깃(Target)**<br>“1시간 후” 혈당(mmol/L) 값입니다. 모델이 예측해야 할 출력 변수로, `test.csv` 에서는 제공되지 않습니다.                   |

- **Lag 변수(72개)**: `bg-6:00` → … → `bg-0:05` → `bg-0:00` 식으로 **과거 최대 6시간**까지 5분 간격 데이터를 나열  
- **모델링 활용**:  
  - **입력(X)**: `bg, insulin, carbs, hr, steps, cals, activity` 의 lag 변수 + `time`, `p_num`, `id`  
  - **출력(y)**: `bg+1:00` (1시간 후 혈당)  
- **분석 예시**:  
  - 최근 30분(`bg-0:30`~`bg-0:00`) 평균·분산 계산 → 혈당 추세 요약 피처 생성  
  - `insulin`, `carbs` lag 합계 → 식후·인슐린 반응 모델링  
  - `hr`, `steps`, `cals` → 활동량-혈당 관계 분석  
  - `activity` → 휴식·운동·수면 중 혈당 패턴 비교  


In [ ]:
na_summary

,category,0,1,2,3,4,5,6,7,8,...,177014,177015,177016,177017,177018,177019,177020,177021,177022,177023
0,activity,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,bg,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,0.666667,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,bg+1:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,cals,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,carbs,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.986111,0.986111,0.986111,...,0.972222,0.972222,0.972222,0.972222,0.972222,0.972222,0.972222,0.972222,0.972222,0.972222
5,hr,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
6,id,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,insulin,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,p_num,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,steps,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
target_stats

,count,mean,std,min,25%,50%,75%,max
bg+1:00,177024.0,8.277045,2.996398,2.2,6.1,7.7,9.9,27.8


### 시각화

In [ ]:
import plotly.express as px

row_id = train["id"].iloc[1]  # 첫 행 사용, 필요시 변경
row = train[train["id"] == row_id].iloc[0]

bg_cols = [c for c in train.columns if c.startswith("bg-")]

offsets = []
values  = []
for col in bg_cols:
    h, m = map(int, col.split("-")[1].split(":"))
    total_min = h*60 + m
    offsets.append(-total_min)  # 현재 시각 기준 과거 offset을 음수로
    values.append(row[col])

df_ts = pd.DataFrame({"minutes_from_now": offsets, "glucose": values})
df_ts = df_ts.sort_values("minutes_from_now")

fig = px.line(
    df_ts,
    x="minutes_from_now",
    y="glucose",
    title=f"CGM Glucose Time Series for {row_id}",
    labels={
        "minutes_from_now": "Minutes from anchor (negative = past)",
        "glucose": "Glucose (mmol/L)"
    }
)
fig.update_traces(mode="markers")
fig.show()


In [ ]:
# 결측치 비율 계산 (%)
na_percent = train.isna().mean() * 100
na_df = na_percent.sort_values(ascending=False).reset_index()
na_df.columns = ["column", "na_percent"]

fig = px.bar(
    na_df,
    x="column",
    y="na_percent",
    title="Missing Percentage by Column",
    labels={"na_percent":"Missing (%)"},
    height=600
)
fig.update_layout(xaxis_tickangle=-90, margin=dict(l=40, r=40, t=80, b=200))
fig.show()


In [ ]:
na_df

,column,na_percent
0,carbs-4:30,98.571945
1,carbs-4:00,98.569685
2,carbs-4:45,98.569121
3,carbs-4:15,98.568556
4,carbs-5:15,98.568556
...,...,...
508,log_bg,0.000000
509,cos24,0.000000
510,sin24,0.000000
511,id,0.000000


---

In [ ]:
cd /content/drive/MyDrive/KSVA_AI/data/brist1d

/content/drive/MyDrive/KSVA_AI/data/brist1d


In [ ]:
sleep = pd.read_csv("iglu_correlations_sleep_wake.csv", engine="python", on_bad_lines="skip", index_col=0)
clin  = pd.read_csv("iglu_and_clinical_measures_correlations.csv", engine="python", on_bad_lines="skip", index_col=0)

In [ ]:
sleep

,X,Y,method,covar,alternative,n,r,CI95%,p-unc,marker,stage,system,ci_low,ci_high,p-fdr
48,ADRR,Waist circumference,pearson,"['age', 'gender']",two-sided,653,0.04829933840998479,[-0.03 0.12],0.21843763420075168,^,Wake,Anthropometry and body composition,-0.03,0.12,0.474082
49,ADRR,Sitting BP diastolic,pearson,"['age', 'gender']",two-sided,653,-0.014603962990973673,[-0.09 0.06],0.7099527922324511,^,Wake,Vascular,-0.09,0.06,0.876222
50,ADRR,Sitting BP systolic,pearson,"['age', 'gender']",two-sided,653,-0.054267606871291195,[-0.13 0.02],0.16667099263196708,^,Wake,Vascular,-0.13,0.02,0.398016
51,ADRR,BMI,pearson,"['age', 'gender']",two-sided,653,0.01851342415455395,[-0.06 0.1 ],0.6372857874513547,^,Wake,Anthropometry and body composition,-0.06,0.10,0.838349
52,ADRR,HDL cholesterol (BT),pearson,"['age', 'gender']",two-sided,250,-0.02172751016326119,[-0.15 0.1 ],0.7334975388939871,^,Wake,Lipids,-0.15,0.10,0.885548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,MODD,CRP (BT),pearson,"['age', 'gender']",two-sided,4,0.27872946021471534,[nan nan],NaN,^,Wake,Inflammatory markers,NaN,NaN,NaN
2883,SDb,GGT (BT),pearson,"['age', 'gender']",two-sided,4,-0.60675103400019,[nan nan],NaN,^,Wake,Liver,NaN,NaN,NaN
2884,SDb,CRP (BT),pearson,"['age', 'gender']",two-sided,4,0.28647195284244037,[nan nan],NaN,^,Wake,Inflammatory markers,NaN,NaN,NaN
2921,SDbdm,GGT (BT),pearson,"['age', 'gender']",two-sided,4,-0.7329491614273104,[nan nan],NaN,^,Wake,Liver,NaN,NaN,NaN


In [ ]:
# Extract unique values
sleep_X = set(sleep["X"].dropna().unique())
clin_X  = set(clin["X"].dropna().unique())
sleep_Y = set(sleep["Y"].dropna().unique())
clin_Y  = set(clin["Y"].dropna().unique())

# Compare X values
all_X = sorted(sleep_X.union(clin_X))
X_cmp = pd.DataFrame({
    "Value": all_X,
    "in_sleep_correlation": [x in sleep_X for x in all_X],
    "in_clinical_correlation": [x in clin_X for x in all_X]
})

# Compare Y values
all_Y = sorted(sleep_Y.union(clin_Y))
Y_cmp = pd.DataFrame({
    "Measure": all_Y,
    "in_sleep_correlation": [y in sleep_Y for y in all_Y],
    "in_clinical_correlation": [y in clin_Y for y in all_Y]
})


## 공통 지표 (수면·임상 상관 테이블 모두 포함)

---

### 1. 백분율 기반 지표 [%]
- **위험 구간 비율**  
  `% above 140`, `% above 180`, `% above 200`, `% above 250`  
- **저혈당 구간 비율**  
  `% below 70`, `% below 54`  
- **표준 구간 비율**  
  `% in 63-140`, `% in 70-180`  

---

### 2. 사분위수·분포 지표
- **사분위수**: `1st quartile`, `Median`, `3rd quartile`  
- **극값·범위**: `Min`, `Max`, `Range`  
- **사분위 범위**: `IQR`  

---

### 3. 기초 통계
- `Mean`, `SD`, `SD.Roc`  

---

### 4. 변동성·위험 지표
- **위험도 지수**: `ADRR`, `COGI`  
- **변동계수**: `CVmean`, `CVsd`  
- **파동 지표**: `M value`, `MAD`, `MAGE`, `MODD`, `MAG`, `J_index`  
- **저·고혈당 위험 지표**: `HBGI`, `LBGI`, `Hyper index`, `Hypo index`, `IGC`  
- **기타 변동 지표**: `CONGA`, `GVP`, `AUC`, `pearson`  

---

### 5. 수면 모니터링 (SM)
- `AHI (SM)`, `ODI (SM)`, `RDI (SM)`  
- `Total Sleep Time (SM)`, `Total Wake Time (SM)`  
- `Mean Oxygen Saturation (SM)`  

---

### 6. 혈액검사 (BT)
- **지질 프로필**:  
  `HDL cholesterol (BT)`, `LDL cholesterol (BT)`, `Total cholesterol (BT)`,  
  `Triglycerides (BT)`  
- **간 효소 등**: `AST\GOT (BT)`, `ALT\GPT (BT)`, `GGT (BT)`  
- **염증·당화 지표**: `CRP (BT)`, `HbA1C (BT)`  
- **기타**: `WBC (BT)`, `eGFR (BT)`  

---

### 7. 생체 계측·혈역학
- `BMI`, `Waist circumference`  
- `Sitting BP systolic`, `Sitting BP diastolic`  

---

### 8. 영상 진단
- **DXA (골밀도)**:  
  `Scanned VAT mass (DXA)`, `Total fat mass (DXA)`, `Android tissue fat percent (DXA)`  
- **초음파 (US)**:  
  `Carotid - intima media thickness (US)`  
  `Liver attenuation (US)`, `Liver elasticity (US)`,  
  `Liver sound speed (US)`, `Liver viscosity (US)`  
- **혈관 영상 (FI)**:  
  `Average width (FI)`, `Fractal dimension (FI)`, `Vessel density (FI)`  

---

### 9. 식사 기록 (DL)
- `Median daily caloric intake (DL)`  
- `Median daily carbohydrate caloric intake (%) (DL)`  
- `Median daily lipid caloric intake (%) (DL)`  
- `Median daily protein caloric intake (%) (DL)`  

---

### 10. 기타 지표
- **상관 지표**: `pearson`  
- **지방 비율**: `vat/fat`  

---

> **in_sleep_correlation** = 해당 지표가 수면 상관 테이블에 있음  
> **in_clinical_correlation** = 해당 지표가 임상 상관 테이블에 있음  


| Measure                                              | 설명                                                                                               | 수면 상관 포함 여부 | 임상 상관 포함 여부 |
|------------------------------------------------------|----------------------------------------------------------------------------------------------------|---------------------|---------------------|
| **AHI (SM)**                                         | 수면 무호흡·저호흡 지수(Apnea–Hypopnea Index)                                                     | ✔︎                  | ✔︎                  |
| **ALT\GPT (BT)**                                     | 혈중 ALT(알라닌 아미노전달효소, GPT) 수치 (Blood Test)                                             | ✔︎                  | ✔︎                  |
| **AST\GOT (BT)**                                     | 혈중 AST(아스파테이트 아미노전달효소, GOT) 수치 (Blood Test)                                        | ✔︎                  | ✔︎                  |
| **Android tissue fat percent (DXA)**                 | DXA(이중 에너지 X선 흡수계측)로 측정한 복부(android) 지방 비율                                      | ✔︎                  | ✔︎                  |
| **Average width (FI)**                               | FI(Fundus Imaging 등)에서 계산된 평균 혈관 폭                                                      | ✔︎                  | ✔︎                  |
| **BMI**                                              | 체질량지수(Body Mass Index)                                                                        | ✔︎                  | ✔︎                  |
| **CRP (BT)**                                         | 혈중 C-반응성 단백질(C-reactive protein) 수치 (Blood Test)                                         | ✔︎                  | ✔︎                  |
| **Carotid - intima media thickness (US)**            | 경동맥 내중막 두께(Carotid IMT), 초음파(US) 측정                                                     | ✔︎                  | ✔︎                  |
| **Fractal dimension (FI)**                           | FI에서 계산된 망상 구조 복잡성(Fractal Dimension)                                                  | ✔︎                  | ✔︎                  |
| **GGT (BT)**                                         | 혈중 GGT(감마-글루타밀 전이효소) 수치 (Blood Test)                                                  | ✔︎                  | ✔︎                  |
| **HDL cholesterol (BT)**                             | 혈중 HDL 콜레스테롤(High-Density Lipoprotein) 수치 (Blood Test)                                    | ✔︎                  | ✔︎                  |
| **HbA1C (BT)**                                       | 혈중 당화혈색소(HbA1c) 비율 (Blood Test)                                                           | ✔︎                  | ✔︎                  |
| **LDL cholesterol (BT)**                             | 혈중 LDL 콜레스테롤(Low-Density Lipoprotein) 수치 (Blood Test)                                      | ✔︎                  | ✔︎                  |
| **Liver attenuation (US)**                           | 초음파 기반 간 감쇠도 지표(Liver Attenuation)                                                        | ✔︎                  | ✔︎                  |
| **Liver elasticity (US)**                            | 초음파 탄성파 기반 간 탄성 지표(Liver Elasticity)                                                   | ✔︎                  | ✔︎                  |
| **Liver sound speed (US)**                           | 초음파에서 측정한 간 조직 내 음속(Liver Sound Speed)                                                | ✔︎                  | ✔︎                  |
| **Liver viscosity (US)**                             | 초음파를 통해 추정한 간 점도 지표(Liver Viscosity)                                                  | ✔︎                  | ✔︎                  |
| **Mean Oxygen Saturation (SM)**                      | 수면 중 평균 산소포화도(SpO₂) (Sleep Monitor)                                                      | ✔︎                  | ✔︎                  |
| **Median daily caloric intake (DL)**                 | 일일 중앙값 칼로리 섭취량(Diet Log)                                                                 | ✔︎                  | ✔︎                  |
| **Median daily carbohydrate caloric intake (%) (DL)**| 일일 중앙값 탄수화물 열량 비율(%) (Diet Log)                                                        | ✔︎                  | ✔︎                  |
| **Median daily lipid caloric intake (%) (DL)**       | 일일 중앙값 지방 열량 비율(%) (Diet Log)                                                            | ✔︎                  | ✔︎                  |
| **Median daily protein caloric intake (%) (DL)**     | 일일 중앙값 단백질 열량 비율(%) (Diet Log)                                                          | ✔︎                  | ✔︎                  |
| **ODI (SM)**                                         | 산소 탈포화 지수(Oxygen Desaturation Index) (Sleep Monitor)                                         | ✔︎                  | ✔︎                  |
| **RDI (SM)**                                         | 호흡 장애 지수(Respiratory Disturbance Index) (Sleep Monitor)                                       | ✔︎                  | ✔︎                  |
| **Scanned VAT mass (DXA)**                           | 스캔된 내장지방(Visceral Adipose Tissue) 질량 (DXA)                                                 | ✔︎                  | ✔︎                  |
| **Sitting BP diastolic**                             | 앉아 있을 때 이완기 혈압(Diastolic Blood Pressure)                                                  | ✔︎                  | ✔︎                  |
| **Sitting BP systolic**                              | 앉아 있을 때 수축기 혈압(Systolic Blood Pressure)                                                   | ✔︎                  | ✔︎                  |
| **Total Sleep Time (SM)**                            | 총 수면 시간(Total Sleep Time) (Sleep Monitor)                                                      | ✔︎                  | ✔︎                  |
| **Total Wake Time (SM)**                             | 총 깨어 있는 시간(Total Wake Time) (Sleep Monitor)                                                   | ✔︎                  | ✔︎                  |
| **Total cholesterol (BT)**                           | 혈중 총콜레스테롤(Total Cholesterol) 수치 (Blood Test)                                               | ✔︎                  | ✔︎                  |
| **Total fat mass (DXA)**                             | 총 체지방 질량(Total Fat Mass) (DXA)                                                                 | ✔︎                  | ✔︎                  |
| **Triglycerides (BT)**                               | 혈중 트리글리세라이드(Triglycerides) 수치 (Blood Test)                                              | ✔︎                  | ✔︎                  |
| **Vessel density (FI)**                              | FI에서 계산된 혈관 밀도(Vessel Density)                                                            | ✔︎                  | ✔︎                  |
| **WBC (BT)**                                         | 혈중 백혈구 수(WBC count) (Blood Test)                                                              | ✔︎                  | ✔︎                  |
| **Waist circumference**                              | 허리 둘레(Waist circumference)                                                                      | ✔︎                  | ✔︎                  |
| **['age', 'gender']**                                | 참가자 연령(age, years) 및 성별(gender, Male/Female)                                                | ✔︎                  | ✔︎                  |
| **eGFR (BT)**                                        | 추정 사구체 여과율(estimated Glomerular Filtration Rate) (Blood Test)                              | ✔︎                  | ✔︎                  |
| **pearson**                                          | FI 또는 기타 시공간 분석에서의 Pearson 지수 (정수 형태 지표)                                        | ✔︎                  | ✔︎                  |
| **vat/fat**                                          | 내장지방 대비 총지방 비율(Visceral-to-Total Fat Ratio)                                               | ✔︎                  | ✔︎                  |
